# Tutorial 4: Visualization
Exporting to Obsidian Canvas.


In [ ]:
import sys
from pathlib import Path
# Ensure src is in python path
project_root = Path.cwd().parent
if str(project_root / 'src') not in sys.path:
    sys.path.append(str(project_root / 'src'))

import logging
logging.basicConfig(level=logging.INFO)
from matome.exporters.obsidian import ObsidianCanvasExporter
from matome.utils.store import DiskChunkStore
# Ideally load an existing tree/store, but we simulate one here
from domain_models.manifest import DocumentTree, SummaryNode, Chunk
from domain_models.types import DIKWLevel


In [ ]:
# Mock Data for Visualization
store = DiskChunkStore()
chunk = Chunk(index=0, text='Some data chunk', start_char_idx=0, end_char_idx=10)
summary = SummaryNode(id='root', text='Wisdom Node', level=1, children_indices=[0], metadata={'dikw_level': 'wisdom'})
store.add_chunks([chunk])
store.add_summaries([summary])
tree = DocumentTree(root_node=summary, leaf_chunk_ids=[0])


In [ ]:
# Export
exporter = ObsidianCanvasExporter()
output_path = project_root / 'summary_kj.canvas'
exporter.export(tree, output_path, store)
print(f'Exported to {output_path}')
